# Retrieval-Augmented Generation (RAG)

In [2]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [8]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook

chroma_client = chromadb.PersistentClient("../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

nutrition_qna = chroma_client.get_collection("nutrition_qna")

In [4]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.




In [5]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

In [10]:
@function_tool
def nutrition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to answer any questions related to nutrition and pragnancy.

    Args:
        query: The question being asked.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the answer .
    """

    results = nutrition_qna.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):        
        formatted_results.append(doc)

    return "The answer is :\n" + "\n".join(formatted_results)

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [ ]:
# calorie_lookup_tool('bananas')

In [11]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful assistant giving out calorie information and nutrition advice.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrtition_qna_tool first to see if there is an answer in the knowledge base.    
    """,
    tools=[calorie_lookup_tool , nutrition_qna_tool],
)

In [14]:
with trace("Calories and Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "What are the best meal choices for pregnant women and how many calories do they have?",
    )
    print(result.final_output)

Key ideas:
- Focus on iron-rich foods (lean red meat, poultry, fish, beans, lentils, fortified cereals, spinach), folate-rich foods (leafy greens, citrus, fortified grains), calcium (dairy or fortified non-dairy), adequate protein, and plenty of fruits, vegetables, and whole grains.
- Add about 200 extra calories per day (varies by trimester and individual needs).

Sample meal choices (with general calories):
- Iron-rich lunch: Lentil and spinach curry with brown rice (about 400–500 kcal)
- Protein-rich snack: Greek yogurt with berries (about 150–200 kcal)
- Calcium-rich option: Fortified yogurt smoothie with milk and a small handful of almonds (about 250–300 kcal)
- Balanced dinner: Grilled chicken or tofu, quinoa, steamed veggies (about 500–600 kcal)

If you want specific meals with exact calorie counts, tell me the foods you like and I can look up precise values.
